In [ ]:
import numpy as np
from keras.applications import vgg16, resnet50
from keras import backend as K
import imageio

In [ ]:
img_width = 224
img_height = 224
symbol = "vgg"

In [ ]:
if symbol == "resnet":
    model = resnet50.ResNet50(include_top=True, weights='imagenet')
    print(model.summary())
elif symbol == "vgg":
    model = vgg16.VGG16(include_top=False, weights='imagenet')
    print(model.summary())

In [ ]:
def deprocess_image(x):
    # mean and std list for channels (Imagenet)
    reverse_mean = [-0.485, -0.456, -0.406]
    reverse_std = [1/0.229, 1/0.224, 1/0.225]
    for c in range(3):
        x[c] /= reverse_std[c]
        x[c] -= reverse_mean[c]
    # Clip between 0 and 1
    x = np.clip(x, 0, 1)
    # Convert to RGB
    x = np.round(x*255)
    # Shape adj. for diff backend
    if K.image_data_format() == 'channels_first':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    # Convert RGB to GBR   
    x = x[..., ::-1]
    return x

In [ ]:
def preprocess_image(x):
    # Expect to be numpy array between 0-1
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    # Normalize the channels
    for channel, _ in enumerate(x):
        x[channel] -= mean[channel]
        x[channel] /= std[channel]
    return x

In [ ]:
def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + 1e-5)

In [ ]:
def get_visuals_for(layer_name, layer_dict, num_to_extract=None):
    kept_filters = []
    layer_output = layer_dict[layer_name].output
    num_filters = int(layer_output.shape[-1])
    # Extract all if not specified
    if num_to_extract is None:
        num_to_extract = num_filters
    # Go through filters
    for filter_index in range(num_filters):
        if K.image_data_format() == 'channels_first':
            loss = K.mean(layer_output[:, filter_index, :, :])
        else:
            loss = K.mean(layer_output[:, :, :, filter_index])
        # Compute gradient
        grads = K.gradients(loss, input_img)[0]
        # Normalise grad
        grads = normalize(grads)
        # Return loss
        iterate = K.function([input_img], [loss, grads])
        # Lr for gradient ascent
        lr = 1.
        # Start with gray image with noise
        if K.image_data_format() == 'channels_first':
            input_img_data = np.random.random((1, 3, img_width, img_height))
        else:
            input_img_data = np.random.random((1, img_width, img_height, 3))
        # Process input
        input_img_data = preprocess_image(input_img_data)
        # Run gradient ascent
        for i in range(50):
            loss_value, grads_value = iterate([input_img_data])
            input_img_data += grads_value * lr
            if loss_value <= 0.:
                break
        # decode the resulting input image
        if loss_value > 0:
            img = deprocess_image(input_img_data[0])
            kept_filters.append((img, loss_value))
            print('Filter %d processed' % (filter_index))
            if len(kept_filters) >= num_to_extract:
                return kept_filters
    # Extracted less than desired
    return kept_filters

In [ ]:
def best_filters(filter_list, num=4):
    filter_list.sort(key=lambda x: x[1], reverse=True)
    return filter_list[:num * num]

In [ ]:
def plot_filters_square(filts, n, name):
    margin = 1
    width = n * img_width + (n - 1) * margin
    height = n * img_height + (n - 1) * margin
    stitched_filters = np.zeros((width, height, 3))
    # fill the picture with our saved filters
    for i in range(n):
        for j in range(n):
            img, loss = filts[i * n + j]
            stitched_filters[(img_width + margin) * i: (img_width + margin) * i + img_width,
                             (img_height + margin) * j: (img_height + margin) * j + img_height, :] = img
    # save the result to disk
    imageio.imwrite('%s_%dx%d.png' % (name, n, n), stitched_filters)

In [ ]:
# input place-holder
input_img = model.input
# get the symbolic outputs of each "key" layer (we gave them unique names).
layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])

In [ ]:
# Possible CNN filters
cnn_filters = [k for k, v in layer_dict.items() if "convolutional" in str(v)]
print(cnn_filters)

In [ ]:
# Select Layer to visualise from above
layer_name = 'block5_conv1'

In [ ]:
# Select how many to plot (will be square)
plot_n = 16
how_many_more_to_collect_before_chopping = 2  # 1, 2, or None

In [ ]:
# Get visualised filter maps (more than amount to plot so that can sort by loss and keep best)
visual_filts = get_visuals_for(layer_name=layer_name, 
                               layer_dict=layer_dict,
                               num_to_extract=how_many_more_to_collect_before_chopping*plot_n)

In [ ]:
# Trim to just best filters
visual_filts_trim = best_filters(visual_filts, num=int(plot_n**(0.5)))
len(visual_filts_trim)

In [ ]:
# Save and plot square
plot_filters_square(filts=visual_filts_trim, n=int(plot_n**(0.5)), name=layer_name)